In [1]:
import json
import boto3
import sagemaker as sage
from sagemaker.pytorch import PyTorch

boto_session = boto3.Session()
sess = sage.Session(boto_session=boto_session)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ayakovyna/.config/sagemaker/config.yaml


ValueError: Must setup local AWS configuration with a region supported by SageMaker.

In [ ]:
iam = boto3.client('iam', region_name='eu-central-1')
role_name = 'flexki-nano-demo-test-role-hamza'
policy = {
    'Statement': [{
        'Action': 'sts:AssumeRole',
        'Effect': 'Allow',
        'Principal': {'Service': 'sagemaker.amazonaws.com'}},
    ],  
    'Version': '2012-10-17'
}

roles = iam.list_roles()
role_arn = None
for role in roles['Roles']:
    if role['RoleName'] == role_name:
        role_arn = role['Arn']
        
if role_arn == None:
    new_role = iam.create_role(
        AssumeRolePolicyDocument=json.dumps(policy),
        Path='/',
        RoleName=role_name,
    )
    role_arn = new_role['Role']['Arn']

    iam.attach_role_policy(
        RoleName=role_name,
        PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess'
    )

    iam.attach_role_policy(
        RoleName=role_name,
        PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess'
    )

In [ ]:
estimator = PyTorch(
    entry_point='sage-entry.py',
    source_dir='./',
    role=role_arn,
    py_version='py38',
    framework_version='2.1.0',
    instance_count=1,
    instance_type='ml.g4dn.xlarge',\
    env={'conda': {'environment.yml': 'environment.yml'}}
)
estimator.fit()